In [1]:
import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from albumentations import HorizontalFlip, VerticalFlip, Rotate
from tqdm import tqdm
import imageio
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
def augment_data(images, masks, save_path, augment=True):
    size = (512, 512)  # Resize all images and masks to this size

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting the name from the image path """
        name = x.split("/")[-1].split(".")[0]

        """ Reading image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)  # Read image
        y = imageio.mimread(y)[0]  # Read mask

        if augment:
            # Apply augmentations: Horizontal Flip, Vertical Flip, and Rotate
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = Rotate(limit=45, p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            # List of original and augmented images and masks
            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]
        else:
            # No augmentation, just save the original image and mask
            X = [x]
            Y = [y]

        # Save images and masks
        index = 0
        for i, m in zip(X, Y):
            # Resize images and masks
            i = cv2.resize(i, size)
            m = cv2.resize(m, size)

            # Create filenames for the augmented data
            tmp_image_name = f"{name}_{index}.png"
            tmp_mask_name = f"{name}_{index}.png"

            # Create paths to save images and masks
            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            # Save images and masks to the directory
            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1

In [3]:
if __name__ == "__main__":
    """ Seeding for reproducibility """
    np.random.seed(42)

    """ Directories for images and masks """
    image_dir = "/kaggle/input/multi-center-polypbd/PolypDB/PolypDB_modality_wise/WLI/images"
    mask_dir = "/kaggle/input/multi-center-polypbd/PolypDB/PolypDB_modality_wise/WLI/masks"
    save_path_train = "/kaggle/working/augmented_data/train"
    save_path_test = "/kaggle/working/augmented_data/test"

    os.makedirs(os.path.join(save_path_train, "image"), exist_ok=True)
    os.makedirs(os.path.join(save_path_train, "mask"), exist_ok=True)
    os.makedirs(os.path.join(save_path_test, "image"), exist_ok=True)
    os.makedirs(os.path.join(save_path_test, "mask"), exist_ok=True)

In [4]:
image_paths = sorted([os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith('.jpg') or img.endswith('.png')])
mask_paths = sorted([os.path.join(mask_dir, mask) for mask in os.listdir(mask_dir) if mask.endswith('.jpg') or mask.endswith('.png')])

In [5]:
train_x, test_x, train_y, test_y = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)

In [6]:
augment_data(train_x, train_y, save_path_train, augment=True)
augment_data(test_x, test_y, save_path_test, augment=False)

100%|██████████| 718/718 [00:36<00:00, 19.47it/s]


In [7]:
class PolypDB(Dataset):
    def __init__(self, images_path, masks_path):
        self.images_path = images_path
        self.masks_path = masks_path
        self.n_samples = len(images_path)
        
    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = image / 255.0 
        image = np.transpose(image, (2, 0, 1))
        image = image.astype(np.float32)
        image = torch.from_numpy(image)
        
        """ Reading masks """
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
        mask = mask / 255.0 
        mask = np.expand_dims(mask, axis=0)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)
        
        return image, mask
    
    def __len__(self):
        return self.n_samples

In [8]:
class conv_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_c)
        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        return x
    

class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        
        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2))
        
    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)
        
        return x, p
    

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        
        self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
        self.conv = conv_block(out_c + out_c, out_c)
        
    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.conv(x)
        
        return x
    

class build_unet(nn.Module):
    def __init__(self):
        super().__init__()
        
        """ Encoder """
        self.e1 = encoder_block(3, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)
        
        """ Bottleneck """
        self.b = conv_block(512, 1024)
        
        """ Decoder """
        self.d1 = decoder_block(1024, 512)
        self.d2 = decoder_block(512, 256)
        self.d3 = decoder_block(256, 128)
        self.d4 = decoder_block(128, 64)
        
        """ Classifier """
        self.outputs = nn.Conv2d(64, 1, kernel_size=1, padding=0)
        
    def forward(self, inputs):
        """ Encoder """
        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)
        
        """ Bottleneck """
        b = self.b(p4)
        
        """ Decoder """
        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)
        
        outputs = self.outputs(d4)
        
        return outputs 



In [9]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)  # Apply sigmoid activation
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

        return 1 - dice

# Dice + Binary Cross-Entropy Loss
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        bce = F.binary_cross_entropy(inputs, targets, reduction='mean')
        dice_bce = bce + dice_loss

        return dice_bce

In [10]:
train_dataset = PolypDB(
        images_path=sorted([os.path.join(save_path_train, "image", img) for img in os.listdir(os.path.join(save_path_train, "image"))]),
        masks_path=sorted([os.path.join(save_path_train, "mask", mask) for mask in os.listdir(os.path.join(save_path_train, "mask"))])
    )
test_dataset = PolypDB(
        images_path=sorted([os.path.join(save_path_test, "image", img) for img in os.listdir(os.path.join(save_path_test, "image"))]),
        masks_path=sorted([os.path.join(save_path_test, "mask", mask) for mask in os.listdir(os.path.join(save_path_test, "mask"))])
    )

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
model = build_unet().to("cuda")  # Use GPU if available
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = DiceBCELoss()
loss_fn = DiceBCELoss()

In [13]:
def iou_score(preds, targets, smooth=1):
    preds = (preds > 0.5).float()  # Apply a threshold to get binary predictions
    targets = targets.float()
    
    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum() - intersection
    
    return (intersection + smooth) / (union + smooth)

In [14]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time // 60)
    elapsed_secs = int(elapsed_time % 60)
    return elapsed_mins, elapsed_secs

In [15]:
def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0
    model.train()
    
    for x, y in tqdm(loader, desc='Training'):
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    epoch_loss /= len(loader)
    return epoch_loss

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0
    total_iou = 0.0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader, desc='Evaluating'):
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()
            
            total_iou += iou_score(y_pred, y)

    epoch_loss /= len(loader)
    average_iou = total_iou / len(loader)
    return epoch_loss, average_iou

# Training Loop
num_epochs = 20  

for epoch in range(num_epochs):
    start_time = time.time()

    # Training and validation
    train_loss = train(model, train_loader, optimizer, loss_fn, device)
    valid_loss, valid_iou = evaluate(model, test_loader, loss_fn, device)

    # Logging the epoch results
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tVal. Loss: {valid_loss:.3f}')
    print(f'\tVal. IoU: {valid_iou:.3f}')

Evaluating: 100%|██████████| 718/718 [00:37<00:00, 19.07it/s]


Epoch: 01 | Time: 23m 44s
	Train Loss: 0.920
	Val. Loss: 0.939
	Val. IoU: 0.249


Evaluating: 100%|██████████| 718/718 [00:36<00:00, 19.69it/s]


Epoch: 02 | Time: 23m 44s
	Train Loss: 0.684
	Val. Loss: 0.657
	Val. IoU: 0.466


Evaluating: 100%|██████████| 718/718 [00:36<00:00, 19.64it/s]


...

Epoch: 18 | Time: 23m 39s
	Train Loss: 0.192
	Val. Loss: 0.434
	Val. IoU: 0.639


Evaluating: 100%|██████████| 718/718 [00:36<00:00, 19.52it/s]


Epoch: 19 | Time: 23m 40s
	Train Loss: 0.186
	Val. Loss: 0.362
	Val. IoU: 0.692


Evaluating: 100%|██████████| 718/718 [00:36<00:00, 19.44it/s]

Epoch: 20 | Time: 23m 42s
	Train Loss: 0.178
	Val. Loss: 0.358
	Val. IoU: 0.695
